# funktioniert nicht so toll

# 1. Prepare Data

In [25]:
# load data
import pandas as pd
df = pd.read_csv("HeteroticOrbifoldMSSMs.csv")
df = df.drop("Unnamed: 0", axis = 1)
print(df.shape)
df.head()

(124941, 14)


,Y0,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10,Y11,Y12,Y13
0,Z2xZ2,7,3,238,47,1,1,9,6,1,6,0,12,42
1,Z2xZ2,7,3,214,47,1,1,5,2,1,6,0,12,50
2,Z2xZ2,7,3,250,51,1,1,7,4,1,8,0,16,48
3,Z2xZ2,7,3,250,35,1,1,7,4,1,10,0,12,46
4,Z2xZ2,7,3,234,51,1,1,7,4,1,6,0,20,42


In [26]:
# transform Y0 to continuous variable and save it as y
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df.Y0)
import numpy as np
print(np.unique(y))
y += 1
print(np.unique(y))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]


In [27]:
# save Y1, Y2, ..., Y13 as X
X = df.iloc[ : , 1: ]
print(X.shape)
X.head()

(124941, 13)


,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10,Y11,Y12,Y13
0,7,3,238,47,1,1,9,6,1,6,0,12,42
1,7,3,214,47,1,1,5,2,1,6,0,12,50
2,7,3,250,51,1,1,7,4,1,8,0,16,48
3,7,3,250,35,1,1,7,4,1,10,0,12,46
4,7,3,234,51,1,1,7,4,1,6,0,20,42


In [28]:
# split in training and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

# 2. Train and Adjust Hyper Parameters

In [29]:
# split trainings set in tr and val set
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 1)

In [46]:
# train xgboost on tr set
from xgboost import XGBRFClassifier
boorf = XGBRFClassifier(max_depth = 300, n_estimators = 1000, subsample = 1,
                        objective = 'multi:softmax', n_jobs = 11, random_state = 87)
boorf.fit(X_tr, y_tr)

XGBRFClassifier(base_score=0.5, colsample_bylevel=1, colsample_bynode=0.8,
                colsample_bytree=1, gamma=0, learning_rate=1, max_delta_step=0,
                max_depth=300, min_child_weight=1, missing=None,
                n_estimators=1000, n_jobs=11, nthread=None,
                objective='multi:softprob', random_state=87, reg_alpha=0,
                reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
                subsample=1, verbosity=1)

In [47]:
# test boo on val set
# compute confusion matrix and accuracy
pred = boorf.predict(X_val)
np.set_printoptions(linewidth=200)            # for nicer print
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_val, pred))
print(sum(y_val == pred)/len(y_val))

[[  84    0    1   54    0    1    2    0   15    0    4    0    1    1]
 [   0   15    3   29    0    0    0    0    1    0    3    0    0   10]
 [   1    1  127  123    0    0    0    0    3    0    0    0    0    0]
 [   6    2   17 7770    1    7   30    0  888    0    4    9    5    6]
 [   0    3    0   73    9    0    2    0   63    0    2    0    0    1]
 [   1    0    0   20    0  462    4    0   40    0    0    0    0    1]
 [   1    0    0  128    0   13  319    0  352    0    1    1    1    0]
 [   0    1    0    9    0    1    0   12    1    0    0    0    0    0]
 [   4    0    3 1004    1   13  103    0 6648    0    2   22    2    9]
 [   0    0    0    3    0    0    0    0    1    1    0    0    0    0]
 [   3    0    1  145    0    1    4    0   17    0   70    0    3    3]
 [   0    0    0    7    0    1    1    0  527    0    0   79    0    0]
 [   1    0    0   62    0    1    0    0   11    0    2    0   64    2]
 [   1    2    2  122    1    2    1    0   22    0

In [38]:
# best parameters:
# max_depth = 300
# subsample = 1
print(sum(y_val == pred)/len(y_val))

0.7954079335701065


# 3. Train Model on Hole Training Set and Test on Test Set